# Predicting SPN 5246 Occurrences

## Introduction
In this notebook, we aim to create a model that predicts the occurrence of SPN 5246 faults in a trucking diagnostics dataset. SPN 5246 signifies a critical situation that can result in truck towing and costs over $4,000. The dataset is from the Big G trucking company and includes several years of onboard computer sensor readings. Our objective is to help the Big G company avoid SPN 5246 codes by using feature engineering, data exploration, and machine learning algorithms to develop an accurate predictive model.

## Libraries

In [3]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
import pickle

## Data Preparation

### Load Data from Pickle File

In [4]:
# Timeseries, equiptmnet, and SPN features
df = pd.read_pickle("../data/ml_model.pkl")
df

EventTimeStamp  AcceleratorPedal  BarometricPressure  \
EquipmentID                                                                    
301         0      2015-05-11 13:11:20              14.4             13.9200   
            1      2015-05-13 08:22:32              89.6             14.4275   
            2      2015-05-18 09:34:05               0.0             14.3550   
            3      2015-05-21 13:57:35              12.0             14.5725   
            4      2015-05-28 13:31:41              67.2             13.9200   
...                                ...               ...                 ...   
310         548843 2018-09-06 00:34:32               0.0             14.2825   
R1762       548844 2015-02-24 13:45:06               0.0             14.0650   
            548845 2015-02-24 15:31:17               0.0             28.2025   
            548846 2015-02-26 13:12:11               0.0             14.2100   
R1764       548847 2015-02-25 06:08:43              21.6             14.4275   

                    CruiseControlActive_False  CruiseControlActive_True  \
EquipmentID                                                               
301         0                             1.0                       0.0   
            1                             1.0                       0.0   
            2                             1.0                       0.0   
            3                             1.0                       0.0   
            4                             1.0                       0.0   
...                                       ...                       ...   
310         548843                        1.0                       0.0   
R1762       548844                        0.0                       1.0   
            548845                        0.0                       2.0   
            548846                        1.0                       0.0   
R1764       548847                        1.0                       0.0   

                    CruiseControlSetSpeed  DistanceLtd  \
EquipmentID                                              
301         0                     0.00000    120841.70   
            1                     0.00000    121687.10   
            2                     0.00000    123057.10   
            3                     0.00000    124496.50   
            4                     0.00000    125989.20   
...                                   ...          ...   
310         548843               64.62260    279964.50   
R1762       548844               66.48672     79796.12   
            548845              132.97344    159709.36   
            548846               66.48672     80838.70   
R1764       548847               66.48672     83681.85   

                    EngineCoolantTemperature  EngineLoad  EngineOilPressure  \
EquipmentID                                                                   
301         0                          185.0        18.0              55.10   
            1                          186.8        38.0              52.78   
            2                          185.0         0.0              55.68   
            3                          183.2        14.0              51.62   
            4                          183.2        68.0              51.62   
...                                      ...         ...                ...   
310         548843                     186.8        10.0              22.04   
R1762       548844                     185.0        83.0              42.34   
            548845                     364.6       136.0              84.68   
            548846                     134.6        11.0              38.86   
R1764       548847                     154.4        12.0              40.60   

                    ...  spn_938  spn_939  spn_94  spn_940  spn_941  spn_95  \
EquipmentID         ...                                                       
301         0       ...      0.0      0.0     0.0      0.0      0.

### Feature Selection

In [6]:
# Select features and target (SPN 5246)
X = df.drop(["spn_5246"], axis=1)
y = df
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 3)

### Apply rolling window


In [8]:
# Resample data on a daily basis
one_hot_fault_daily = df.groupby(by='EquipmentID').resample('D').sum(numeric_only=True)

# Drop all rows containing NaN values
one_hot_fault_daily.dropna(inplace=True)

ValueError: MultiIndex has no single backing array. Use 'MultiIndex.to_numpy()' to get a NumPy array of tuples.

In [ ]:
one_hot_fault_daily

SPN_0  SPN_16  SPN_27  SPN_33  SPN_37  SPN_38  \
EquipmentID EventTimeStamp                                                  
301         2015-05-11          0       0       0       0       0       0   
            2015-05-12          0       0       0       0       0       0   
            2015-05-13          0       0       0       0       0       0   
            2015-05-14          0       0       0       0       0       0   
            2015-05-15          0       0       0       0       0       0   
...                           ...     ...     ...     ...     ...     ...   
310         2018-09-06          0       0       0       0       0       0   
R1762       2015-02-24          0       0       0       0       0       0   
            2015-02-25          0       0       0       0       0       0   
            2015-02-26          0       0       0       0       0       0   
R1764       2015-02-25          0       0       0       0       0       0   

                            SPN_51  SPN_70  SPN_74  SPN_75  ...  SPN_520413  \
EquipmentID EventTimeStamp                                  ...               
301         2015-05-11           0       0       0       0  ...           0   
            2015-05-12           0       0       0       0  ...           0   
            2015-05-13           0       0       0       0  ...           0   
            2015-05-14           0       0       0       0  ...           0   
            2015-05-15           0       0       0       0  ...           0   
...                            ...     ...     ...     ...  ...         ...   
310         2018-09-06           0       0       0       0  ...           0   
R1762       2015-02-24           0       0       0       0  ...           0   
            2015-02-25           0       0       0       0  ...           0   
            2015-02-26           0       0       0       0  ...           0   
R1764       2015-02-25           0       0       0       0  ...           0   

                            SPN_520953  SPN_521032  SPN_523530  SPN_523531  \
EquipmentID EventTimeStamp                                                   
301         2015-05-11               0           0           0           0   
            2015-05-12               0           0           0           0   
            2015-05-13               0           0           0           0   
            2015-05-14               0           0           0           0   
            2015-05-15               0           0           0           0   
...                                ...         ...         ...         ...   
310         2018-09-06               0           0           0           0   
R1762       2015-02-24               0           0           0           0   
            2015-02-25               0           0           0           0   
            2015-02-26               0           0           0           0   
R1764       2015-02-25               0           0           0           0   

                            SPN_523543  SPN_524033  SPN_524037  SPN_524071  \
EquipmentID EventTimeStamp                                                   
301         2015-05-11               0           0           0           0   
            2015-05-12               0           0           0           0   
            2015-05-13               0           0           0           0   
            2015-05-14               0           0           0           0   
            2015-05-15               0           0           0           0   
...                                ...         ...         ...         ...   
310         2018-09-06               0           0           0           0   
R1762       2015-02-24               0           0           0           0   
            2015-02-25               0           0           0           0   
            2015-02-26               0           0           0           0   
R1764       2015-02-25               0         

## Model Training and Evaluation

### Scale and Train/Test split the Data

In [7]:
# Scale the data
scaler = StandardScaler()

# Train logistic regression model
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'n

### Train Logistic Regression Model

In [ ]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

ValueError: could not convert string to float: 'R1762'

### Evaluate the model

In [ ]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
probs = clf.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend()
plt.show()